# Lab 3-2: 모니터링 시스템 대화형 실습

Jupyter Notebook을 통해 Prometheus 메트릭과 모니터링 시스템을 대화형으로 학습합니다.

## 📋 실습 내용

1. Prometheus 메트릭 생성 및 수집
2. Custom Metrics Exporter 구현
3. PromQL 쿼리 실습
4. 메트릭 시각화
5. A/B 테스트 시뮬레이션
6. 알림 규칙 테스트

---

## 1. 환경 설정

필요한 패키지를 설치하고 import합니다.

In [ ]:
# 패키지 설치
!pip install prometheus-client requests pandas numpy matplotlib seaborn -q

print("✅ 패키지 설치 완료!")

In [ ]:
import time
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from prometheus_client import start_http_server, Gauge, Counter, Histogram, Summary
from datetime import datetime, timedelta
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

# Plot 스타일 설정
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✅ Import 완료")

---

## 2. Prometheus 메트릭 타입 이해

Prometheus는 4가지 주요 메트릭 타입을 제공합니다.

### 2.1 Counter (카운터)

- 단조 증가하는 값 (0에서 시작, 증가만 가능)
- 예: 요청 수, 에러 수, 완료된 작업 수

In [ ]:
# Counter 생성
prediction_counter = Counter(
    'model_predictions_total',
    'Total number of predictions',
    ['model_version', 'status']
)

# Counter 증가
print("=" * 60)
print("Counter 실습")
print("=" * 60)

for i in range(10):
    # 성공 예측
    if random.random() > 0.1:  # 90% 성공
        prediction_counter.labels(model_version='v1.0', status='success').inc()
    else:  # 10% 실패
        prediction_counter.labels(model_version='v1.0', status='error').inc()
    
    time.sleep(0.1)

print("\n✅ Counter 증가 완료")
print(f"현재 값은 /metrics 엔드포인트에서 확인 가능합니다")

### 2.2 Gauge (게이지)

- 증가/감소 모두 가능한 값
- 예: CPU 사용률, 메모리 사용량, 현재 활성 사용자 수

In [ ]:
# Gauge 생성
mae_gauge = Gauge(
    'model_mae_score',
    'Current MAE score of the model',
    ['model_version']
)

r2_gauge = Gauge(
    'model_r2_score',
    'Current R² score of the model',
    ['model_version']
)

# Gauge 값 시뮬레이션
print("=" * 60)
print("Gauge 실습 - 모델 성능 메트릭 시뮬레이션")
print("=" * 60)

mae_values = []
r2_values = []

for i in range(20):
    # v1.0 모델 - 안정적
    mae_v1 = 0.445 + random.gauss(0, 0.01)
    r2_v1 = 0.798 + random.gauss(0, 0.005)
    
    # v2.0 모델 - 더 좋은 성능
    mae_v2 = 0.362 + random.gauss(0, 0.01)
    r2_v2 = 0.885 + random.gauss(0, 0.005)
    
    # Gauge 업데이트
    mae_gauge.labels(model_version='v1.0').set(mae_v1)
    r2_gauge.labels(model_version='v1.0').set(r2_v1)
    mae_gauge.labels(model_version='v2.0').set(mae_v2)
    r2_gauge.labels(model_version='v2.0').set(r2_v2)
    
    mae_values.append({'v1.0': mae_v1, 'v2.0': mae_v2})
    r2_values.append({'v1.0': r2_v1, 'v2.0': r2_v2})
    
    if (i + 1) % 5 == 0:
        print(f"Step {i+1:2d} | v1.0: MAE={mae_v1:.4f}, R²={r2_v1:.4f} | v2.0: MAE={mae_v2:.4f}, R²={r2_v2:.4f}")
    
    time.sleep(0.1)

print("\n✅ Gauge 업데이트 완료")

### 2.3 Histogram (히스토그램)

- 값의 분포를 측정 (버킷별 빈도)
- 예: 응답 시간, 요청 크기

In [ ]:
# Histogram 생성
latency_histogram = Histogram(
    'model_prediction_latency_seconds',
    'Prediction latency in seconds',
    ['model_version'],
    buckets=(0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5, 1.0, 2.5, 5.0)
)

# Latency 시뮬레이션
print("=" * 60)
print("Histogram 실습 - 예측 레이턴시 측정")
print("=" * 60)

latencies_v1 = []
latencies_v2 = []

for i in range(100):
    # v1.0: 평균 50ms, 더 큰 변동
    latency_v1 = abs(random.gauss(0.05, 0.02))
    latency_histogram.labels(model_version='v1.0').observe(latency_v1)
    latencies_v1.append(latency_v1)
    
    # v2.0: 평균 40ms, 더 안정적
    latency_v2 = abs(random.gauss(0.04, 0.01))
    latency_histogram.labels(model_version='v2.0').observe(latency_v2)
    latencies_v2.append(latency_v2)

print(f"\n📊 v1.0 레이턴시: 평균 {np.mean(latencies_v1)*1000:.2f}ms, p95 {np.percentile(latencies_v1, 95)*1000:.2f}ms")
print(f"📊 v2.0 레이턴시: 평균 {np.mean(latencies_v2)*1000:.2f}ms, p95 {np.percentile(latencies_v2, 95)*1000:.2f}ms")
print("\n✅ Histogram 관측 완료")

### 2.4 Summary (요약)

- Histogram과 유사하지만 분위수 계산
- 예: 요청 지속 시간, 응답 크기

In [ ]:
# Summary 생성
request_size_summary = Summary(
    'model_request_size_bytes',
    'Size of prediction requests in bytes',
    ['model_version']
)

# Request size 시뮬레이션
print("=" * 60)
print("Summary 실습 - 요청 크기 측정")
print("=" * 60)

for i in range(50):
    # 100-500 bytes
    size_v1 = random.randint(100, 500)
    request_size_summary.labels(model_version='v1.0').observe(size_v1)
    
    size_v2 = random.randint(100, 500)
    request_size_summary.labels(model_version='v2.0').observe(size_v2)

print("\n✅ Summary 관측 완료")

---

## 3. Prometheus HTTP Server 시작

메트릭을 `/metrics` 엔드포인트로 노출합니다.

In [ ]:
# Prometheus HTTP Server 시작
PORT = 8000

try:
    start_http_server(PORT)
    print("=" * 60)
    print("✅ Prometheus Metrics Server 시작!")
    print("=" * 60)
    print(f"\n📡 Metrics endpoint: http://localhost:{PORT}/metrics")
    print("\n새 터미널에서 확인:")
    print(f"  curl http://localhost:{PORT}/metrics")
    print("\n또는 브라우저에서 접속하세요!")
except OSError:
    print(f"⚠️  포트 {PORT}이(가) 이미 사용 중입니다")
    print("   Metrics server가 이미 실행 중일 수 있습니다")

---

## 4. 메트릭 시각화

수집된 메트릭을 시각화합니다.

In [ ]:
# MAE 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# MAE 비교
mae_df = pd.DataFrame(mae_values)
mae_df.plot(ax=ax1, title='Model MAE Over Time', ylabel='MAE', xlabel='Step')
ax1.legend(['v1.0', 'v2.0'])
ax1.grid(True, alpha=0.3)

# R² 비교
r2_df = pd.DataFrame(r2_values)
r2_df.plot(ax=ax2, title='Model R² Score Over Time', ylabel='R² Score', xlabel='Step')
ax2.legend(['v1.0', 'v2.0'])
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 모델 성능 비교:")
print(f"  v1.0: MAE={mae_df['v1.0'].mean():.4f}, R²={r2_df['v1.0'].mean():.4f}")
print(f"  v2.0: MAE={mae_df['v2.0'].mean():.4f}, R²={r2_df['v2.0'].mean():.4f}")
print(f"\n✨ v2.0이 MAE {(1 - mae_df['v2.0'].mean()/mae_df['v1.0'].mean())*100:.1f}% 개선!")

In [ ]:
# Latency 분포 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax1.hist(latencies_v1, bins=30, alpha=0.7, label='v1.0', color='blue')
ax1.hist(latencies_v2, bins=30, alpha=0.7, label='v2.0', color='orange')
ax1.set_xlabel('Latency (seconds)')
ax1.set_ylabel('Frequency')
ax1.set_title('Prediction Latency Distribution')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Box plot
latency_df = pd.DataFrame({'v1.0': latencies_v1, 'v2.0': latencies_v2})
latency_df.boxplot(ax=ax2)
ax2.set_ylabel('Latency (seconds)')
ax2.set_title('Latency Comparison')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 레이턴시 통계:")
print("\nv1.0:")
print(f"  평균: {np.mean(latencies_v1)*1000:.2f}ms")
print(f"  중앙값: {np.median(latencies_v1)*1000:.2f}ms")
print(f"  p95: {np.percentile(latencies_v1, 95)*1000:.2f}ms")
print(f"  p99: {np.percentile(latencies_v1, 99)*1000:.2f}ms")

print("\nv2.0:")
print(f"  평균: {np.mean(latencies_v2)*1000:.2f}ms")
print(f"  중앙값: {np.median(latencies_v2)*1000:.2f}ms")
print(f"  p95: {np.percentile(latencies_v2, 95)*1000:.2f}ms")
print(f"  p99: {np.percentile(latencies_v2, 99)*1000:.2f}ms")

---

## 5. PromQL 쿼리 실습

Prometheus 쿼리 언어(PromQL)를 사용하여 메트릭을 조회합니다.

In [ ]:
# Prometheus 연결 설정
PROMETHEUS_URL = "http://localhost:9090"  # Prometheus 서버 주소

def query_prometheus(query):
    """Prometheus 쿼리 실행"""
    try:
        response = requests.get(
            f"{PROMETHEUS_URL}/api/v1/query",
            params={'query': query}
        )
        if response.status_code == 200:
            return response.json()
        else:
            return {'error': f'HTTP {response.status_code}'}
    except Exception as e:
        return {'error': str(e)}

print("=" * 60)
print("PromQL 쿼리 함수 준비 완료")
print("=" * 60)
print(f"\n⚠️  주의: Prometheus가 {PROMETHEUS_URL}에서 실행 중이어야 합니다")
print("\nPrometheus 연결:")
print("  kubectl port-forward -n monitoring-system svc/prometheus-server 9090:80")

In [ ]:
# 예제 쿼리들
queries = [
    ("현재 MAE 값", 'model_mae_score'),
    ("현재 R² 값", 'model_r2_score'),
    ("총 예측 수", 'model_predictions_total'),
    ("최근 5분 예측 증가율", 'rate(model_predictions_total[5m])'),
    ("p95 레이턴시", 'histogram_quantile(0.95, rate(model_prediction_latency_seconds_bucket[5m]))'),
]

print("=" * 60)
print("PromQL 쿼리 예제")
print("=" * 60)

for description, query in queries:
    print(f"\n📊 {description}")
    print(f"   Query: {query}")
    
    result = query_prometheus(query)
    
    if 'error' in result:
        print(f"   ⚠️  Error: {result['error']}")
    elif result.get('status') == 'success':
        data = result.get('data', {}).get('result', [])
        if data:
            for item in data[:3]:  # 최대 3개만 표시
                labels = item.get('metric', {})
                value = item.get('value', [None, 'N/A'])[1]
                print(f"   ✅ {labels}: {value}")
        else:
            print("   ℹ️  No data")
    else:
        print(f"   ⚠️  Unexpected response: {result}")

---

## 6. A/B 테스트 시뮬레이션

두 모델 버전의 성능을 비교합니다.

In [ ]:
print("=" * 60)
print("A/B 테스트 시뮬레이션 시작")
print("=" * 60)

# 트래픽 분할 설정
traffic_split = {
    'v1.0': 0.5,  # 50%
    'v2.0': 0.5   # 50%
}

print(f"\n트래픽 분할: v1.0={traffic_split['v1.0']*100:.0f}%, v2.0={traffic_split['v2.0']*100:.0f}%")

# 시뮬레이션 실행
n_requests = 200
results = {'v1.0': [], 'v2.0': []}

print(f"\n{n_requests}개 요청 시뮬레이션 중...")

for i in range(n_requests):
    # 트래픽 분할
    version = 'v1.0' if random.random() < traffic_split['v1.0'] else 'v2.0'
    
    # 성능 시뮬레이션
    if version == 'v1.0':
        mae = 0.445 + random.gauss(0, 0.01)
        latency = abs(random.gauss(0.05, 0.02))
        success = random.random() > 0.05  # 95% 성공률
    else:  # v2.0
        mae = 0.362 + random.gauss(0, 0.01)
        latency = abs(random.gauss(0.04, 0.01))
        success = random.random() > 0.02  # 98% 성공률
    
    # 결과 기록
    results[version].append({
        'mae': mae,
        'latency': latency,
        'success': success
    })
    
    # 메트릭 업데이트
    mae_gauge.labels(model_version=version).set(mae)
    latency_histogram.labels(model_version=version).observe(latency)
    status = 'success' if success else 'error'
    prediction_counter.labels(model_version=version, status=status).inc()
    
    if (i + 1) % 50 == 0:
        print(f"  진행: {i+1}/{n_requests}")

print("\n✅ 시뮬레이션 완료!")

In [ ]:
# A/B 테스트 결과 분석
print("=" * 60)
print("A/B 테스트 결과 분석")
print("=" * 60)

for version in ['v1.0', 'v2.0']:
    data = results[version]
    
    maes = [d['mae'] for d in data]
    latencies = [d['latency'] * 1000 for d in data]  # ms로 변환
    success_rate = sum(d['success'] for d in data) / len(data) * 100
    
    print(f"\n{'='*30}")
    print(f"모델 버전: {version}")
    print(f"{'='*30}")
    print(f"요청 수: {len(data)}")
    print(f"\n📊 성능 메트릭:")
    print(f"  MAE:")
    print(f"    평균: {np.mean(maes):.4f}")
    print(f"    표준편차: {np.std(maes):.4f}")
    print(f"\n  레이턴시:")
    print(f"    평균: {np.mean(latencies):.2f}ms")
    print(f"    p50: {np.percentile(latencies, 50):.2f}ms")
    print(f"    p95: {np.percentile(latencies, 95):.2f}ms")
    print(f"    p99: {np.percentile(latencies, 99):.2f}ms")
    print(f"\n  성공률: {success_rate:.2f}%")

In [ ]:
# A/B 테스트 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# MAE 비교
maes_v1 = [d['mae'] for d in results['v1.0']]
maes_v2 = [d['mae'] for d in results['v2.0']]
axes[0, 0].boxplot([maes_v1, maes_v2], labels=['v1.0', 'v2.0'])
axes[0, 0].set_title('MAE Comparison')
axes[0, 0].set_ylabel('MAE')
axes[0, 0].grid(True, alpha=0.3)

# Latency 비교
latencies_v1 = [d['latency'] * 1000 for d in results['v1.0']]
latencies_v2 = [d['latency'] * 1000 for d in results['v2.0']]
axes[0, 1].boxplot([latencies_v1, latencies_v2], labels=['v1.0', 'v2.0'])
axes[0, 1].set_title('Latency Comparison')
axes[0, 1].set_ylabel('Latency (ms)')
axes[0, 1].grid(True, alpha=0.3)

# Success Rate 비교
success_v1 = sum(d['success'] for d in results['v1.0']) / len(results['v1.0']) * 100
success_v2 = sum(d['success'] for d in results['v2.0']) / len(results['v2.0']) * 100
axes[1, 0].bar(['v1.0', 'v2.0'], [success_v1, success_v2], color=['blue', 'orange'])
axes[1, 0].set_title('Success Rate Comparison')
axes[1, 0].set_ylabel('Success Rate (%)')
axes[1, 0].set_ylim([90, 100])
axes[1, 0].grid(True, alpha=0.3)

# Request Distribution
request_counts = [len(results['v1.0']), len(results['v2.0'])]
axes[1, 1].pie(request_counts, labels=['v1.0', 'v2.0'], autopct='%1.1f%%', startangle=90)
axes[1, 1].set_title('Request Distribution')

plt.tight_layout()
plt.show()

# 승자 결정
print("\n" + "="*60)
print("🏆 A/B 테스트 결론")
print("="*60)

mae_improvement = (1 - np.mean(maes_v2) / np.mean(maes_v1)) * 100
latency_improvement = (1 - np.mean(latencies_v2) / np.mean(latencies_v1)) * 100

print(f"\nv2.0이 v1.0 대비:")
print(f"  📊 MAE: {mae_improvement:+.2f}% 개선")
print(f"  ⚡ 레이턴시: {latency_improvement:+.2f}% 개선")
print(f"  ✅ 성공률: {success_v2 - success_v1:+.2f}%p 개선")

if mae_improvement > 0 and latency_improvement > 0:
    print("\n🎉 결론: v2.0을 프로덕션에 배포하는 것을 권장합니다!")
else:
    print("\n⚠️  결론: 추가 분석이 필요합니다")

---

## 7. 알림 규칙 시뮬레이션

성능 임계값을 설정하고 알림 조건을 테스트합니다.

In [ ]:
# 알림 규칙 정의
alert_rules = {
    'HighMAE': {
        'threshold': 0.50,
        'metric': 'mae',
        'description': 'MAE가 0.50을 초과'
    },
    'LowR2': {
        'threshold': 0.75,
        'metric': 'r2',
        'description': 'R²가 0.75 미만'
    },
    'HighLatency': {
        'threshold': 100,  # ms
        'metric': 'latency',
        'description': '레이턴시가 100ms 초과'
    },
    'LowSuccessRate': {
        'threshold': 95,  # %
        'metric': 'success_rate',
        'description': '성공률이 95% 미만'
    }
}

print("=" * 60)
print("알림 규칙 설정")
print("=" * 60)

for alert_name, rule in alert_rules.items():
    print(f"\n📢 {alert_name}")
    print(f"   조건: {rule['description']}")
    print(f"   임계값: {rule['threshold']}")

In [ ]:
# 알림 체크 함수
def check_alerts(results, alert_rules):
    """알림 조건 체크"""
    alerts = []
    
    for version, data in results.items():
        # 메트릭 계산
        maes = [d['mae'] for d in data]
        latencies = [d['latency'] * 1000 for d in data]
        success_rate = sum(d['success'] for d in data) / len(data) * 100
        
        metrics = {
            'mae': np.mean(maes),
            'latency': np.percentile(latencies, 95),  # p95
            'success_rate': success_rate
        }
        
        # 알림 체크
        for alert_name, rule in alert_rules.items():
            metric_name = rule['metric']
            threshold = rule['threshold']
            
            if metric_name == 'r2':
                continue  # R² 메트릭은 여기서 계산 안함
            
            value = metrics.get(metric_name)
            
            # 알림 조건 확인
            triggered = False
            if metric_name == 'success_rate':
                triggered = value < threshold
            else:
                triggered = value > threshold
            
            if triggered:
                alerts.append({
                    'alert': alert_name,
                    'version': version,
                    'metric': metric_name,
                    'value': value,
                    'threshold': threshold,
                    'description': rule['description']
                })
    
    return alerts

# 알림 체크 실행
print("=" * 60)
print("알림 조건 체크")
print("=" * 60)

alerts = check_alerts(results, alert_rules)

if alerts:
    print(f"\n🚨 {len(alerts)}개 알림 발생!\n")
    for alert in alerts:
        print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print(f"🔔 {alert['alert']}")
        print(f"   버전: {alert['version']}")
        print(f"   조건: {alert['description']}")
        print(f"   현재 값: {alert['value']:.2f}")
        print(f"   임계값: {alert['threshold']}")
else:
    print("\n✅ 모든 메트릭이 정상 범위 내에 있습니다!")

---

## 8. 실습 완료!

### 학습한 내용

- ✅ Prometheus 메트릭 타입 (Counter, Gauge, Histogram, Summary)
- ✅ Custom Metrics Exporter 구현
- ✅ PromQL 쿼리 작성
- ✅ 메트릭 시각화 (Matplotlib)
- ✅ A/B 테스트 시뮬레이션
- ✅ 알림 규칙 설정 및 테스트

### 다음 단계

1. **Grafana 대시보드 구성**
   - 메트릭을 Grafana에서 시각화
   - 실시간 모니터링 대시보드 생성

2. **Alertmanager 설정**
   - 알림 라우팅 구성
   - Slack/Email 연동

3. **프로덕션 배포**
   - Kubernetes에 모니터링 스택 배포
   - 실제 모델 서빙 연동

---

## 📚 참고 자료

- [Prometheus Python Client](https://github.com/prometheus/client_python)
- [PromQL Documentation](https://prometheus.io/docs/prometheus/latest/querying/basics/)
- [Grafana Dashboards](https://grafana.com/docs/grafana/latest/dashboards/)
- [Alertmanager Configuration](https://prometheus.io/docs/alerting/latest/configuration/)

---

© 2024 현대오토에버 MLOps Training - Lab 3-2